In [1]:
import sys
import csv
import unicodedata
import numpy as np
import pandas as pd

In [2]:
from pymystem3 import Mystem
from collections import Counter
from multiprocessing import Pool
from IPython.display import display
from ipywidgets import FloatProgress
from sklearn.pipeline import Pipeline
from parsers.text_utils import DefaultTextProcessor, Lemmatizer

In [4]:
csv.field_size_limit(sys.maxsize)

9223372036854775807

Разобьём процесс на две части — токенизацию документов (без фильтрации) и, собственно, лемматизацию.

In [29]:
%%time

# Проделаем токенизацию с сохранением промежуточного
# состояния в ruwiki.tonekized.csv.tmp

tokenizer = DefaultTextProcessor()

# 1361758 — предподсчитанное кол-во документов
f = FloatProgress(min=0, max=1361758)
display(f)

unused_char = '\U00037b84'
def strip_accents(s):
    s = s.replace("й", unused_char)
    return "".join((c for c in unicodedata.normalize("NFD", s) if unicodedata.category(c) != "Mn")).replace(unused_char, "й")

def remove_underscores(s):
    return s.replace("_", "")

with open("../datasets/ruwiki/ruwiki.plain.csv", "r") as infile:
    with open("ruwiki.tonekized.csv.tmp", "w") as outfile:
        reader = csv.reader(infile)
        writer = csv.writer(outfile)
        count = 0
        cached_rows = []
        for title, text in reader:
            text = strip_accents(text)
            text = remove_underscores(text)
            tokens = tokenizer.fit_transform(text)
            cached_rows.append((title, " ".join(tokens)))
            count += 1
            if count % 1000 == 0:
                writer.writerows(cached_rows)
                outfile.flush()
                f.value += len(cached_rows)
                cached_rows = []
        # Запишем оставшиеся строчки
        writer.writerows(cached_rows)
        f.value += len(cached_rows)

CPU times: user 40min 14s, sys: 29.9 s, total: 40min 44s
Wall time: 40min 44s


In [30]:
%%time

# Теперь сделаем лемматизацию всех документов при помощи pymystem3
# Распараллеливая процесс на N_PROCS процессоров

N_PROCS = 4

# 1361758 — предподсчитанное кол-во документов
f = FloatProgress(min=0, max=1361758)
display(f)

m = Mystem()

def lemmatize(text):
    return "".join(m.lemmatize(text)).strip()

with open("ruwiki.tonekized.csv.tmp", "r") as infile:
    with open("../datasets/ruwiki/ruwiki.lemmatized.csv", "w") as outfile:
        reader = csv.reader(infile)
        writer = csv.writer(outfile)
        count = 0
        cached_titles = []
        cached_texts = []
        for title, text in reader:
            cached_titles.append(title)
            cached_texts.append(text)
            count += 1
            if count % 1000 == 0:
                with Pool(N_PROCS) as p:
                    lemmatized_texts = p.map(lemmatize, cached_texts)
                writer.writerows(zip(cached_titles, lemmatized_texts))
                outfile.flush()
                f.value += len(cached_titles)
                cached_texts = []
                cached_titles = []
        # Запишем оставшиеся строчки
        with Pool(N_PROCS) as p:
            lemmatized_texts = p.map(lemmatize, cached_texts)
        writer.writerows(zip(cached_titles, lemmatized_texts))
        f.value += len(cached_titles)

CPU times: user 5min 17s, sys: 2min 17s, total: 7min 35s
Wall time: 3h 13min 15s


Посчитаем размер словаря, из которого состоит неотфильтрованная лемматизированная выборка.

In [31]:
%%time

dictionary = Counter()

# 1361758 — предподсчитанное кол-во документов
f = FloatProgress(min=0, max=1361758)
display(f)

with open("../datasets/ruwiki/ruwiki.lemmatized.csv", "r") as infile:
    reader = csv.reader(infile)
    for title, text in reader:
        tokens = text.split()
        dictionary.update(tokens)
        f.value += 1

CPU times: user 31min 53s, sys: 2min 5s, total: 33min 59s
Wall time: 31min 53s


In [32]:
dict_series = pd.DataFrame.from_dict(dictionary, orient="index")[0]

In [33]:
len(dict_series)

3924272

In [34]:
(dict_series > 10).value_counts()[True]

533185

Будем считать, что лемма должна встретиться более 10 раз в коллекции, чтобы мы положили её в словарь. Это сократит размер словаря в 8 раз от первоначального объёма.

Посмотрим на топ-50 слов:

In [35]:
top50_words = dict_series.sort_values(ascending=False)[:50].index
top50_words

Index(['в', 'и', 'год', 'на', 'с', 'быть', 'по', 'из', 'он', 'который', 'а',
       'к', 'не', 'что', 'от', 'для', 'за', '1', 'как', 'этот', 'свой', '2',
       'также', 'до', 'первый', 'время', 'о', 'его', 'после', 'они', '3',
       'район', 'один', 'то', 'становиться', 'при', 'г', 'город', '5',
       'примечание', 'ссылка', 'человек', 'м', 'тот', 'область', 'во', 'это',
       'она', 'весь', 'но'],
      dtype='object')

Слов, которые могли бы иметь выраженную тематику, здесь почти нет, зато довольно много мусорных и общих слов. Будем выбрасывать слова, входящие в этот список, из документов.

Также будем фильтровать слова по стоп-словарю.

In [36]:
stop_words = set(map(str.strip, open("../datasets/ruwiki/stopwords.txt").readlines()))

In [37]:
%%time

common_words = dict_series[dict_series > 10].index

# 1361758 — предподсчитанное кол-во документов
f = FloatProgress(min=0, max=1361758)
display(f)

def accept_word(w):
    return w not in stop_words and w not in top50_words and w in common_words

with open("../datasets/ruwiki/ruwiki.lemmatized.csv", "r") as infile:
    with open("../datasets/ruwiki/ruwiki.filtered.csv", "w") as outfile:
        reader = csv.reader(infile)
        writer = csv.writer(outfile)
        for title, text in reader:
            tokens = text.split()
            writer.writerow((title, " ".join(filter(accept_word, tokens))))
            f.value += 1

CPU times: user 43min 12s, sys: 2min 23s, total: 45min 35s
Wall time: 43min 6s


Наконец, превратим коллекцию с отфильтрованным словарём в файл UCI Bag-of-words.

Для начала построим словарь по отфильтрованной коллекции.

In [38]:
%%time

dictionary = set()
bow_length = 0

# 1361758 — предподсчитанное кол-во документов
f = FloatProgress(min=0, max=1361758)
display(f)

with open("../datasets/ruwiki/ruwiki.filtered.csv", "r") as infile:
    reader = csv.reader(infile)
    for title, text in reader:
        tokens = set(text.split())
        dictionary.update(tokens)
        bow_length += len(tokens)
        f.value += 1

CPU times: user 29min 24s, sys: 2min 3s, total: 31min 27s
Wall time: 29min 25s


In [39]:
len(dictionary)

532345

In [40]:
bow_length

185333372

Запишем словарь в файл и переконвертируем документы.

In [41]:
dict_mapping = dict(zip(dictionary, range(len(dictionary))))
dict_ordering = sorted(zip(range(len(dictionary)), dictionary))

In [42]:
%%time

with open("../datasets/ruwiki/vocab.ruwiki.csv", "w") as dictfile:
    for _, word in dict_ordering:
        dictfile.write("%s text\n" % word)

CPU times: user 382 ms, sys: 11 ms, total: 393 ms
Wall time: 393 ms


In [47]:
%%time

# 1361758 — предподсчитанное кол-во документов
doc_count = 1361758
f = FloatProgress(min=0, max=doc_count)
display(f)

with open("../datasets/ruwiki/docword.ruwiki.csv", "w") as docwordfile:
    docwordfile.write("%d\n%d\n%d\n" % (len(dictionary), doc_count, bow_length))
    with open("../datasets/ruwiki/ruwiki.filtered.csv", "r") as infile:
        reader = csv.reader(infile)
        for docID, (title, text) in enumerate(reader):
            for word, count in Counter(text.split()).items():
                docwordfile.write("%d %d %d\n" % (docID + 1, dict_mapping[word], count))
            f.value += 1

CPU times: user 35min 31s, sys: 2min 22s, total: 37min 53s
Wall time: 35min 38s


---